# Load dataset

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
from sentence_transformers import SentenceTransformer

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
vietnamese_history_dataset_path = "../../datasets/vietnamese_history_dataset/data.json"
df = pd.read_json(vietnamese_history_dataset_path)
df.head()

,title,content,type
0,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,chapter
1,THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA\nBài: THỜI NG...,lesson
2,Những dấu tích của Người tối cổ được tìm thấy ...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nNhững ...,title
3,"Ở giai đoạn đầu, Người tinh khôn sống như thế ...",Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nỞ giai...,title
4,Giai đoạn phát triển của Người tinh khôn có gì...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nGiai đ...,title


In [16]:
df["content_embedding"] = None
df["embeded"] = None

for idx in range(len(df)):
    df["content_embedding"] = df["title"][idx] + " " + df["content"][idx]

df.head()

,title,content,type,content_embedding,embeded
0,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,chapter,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,None
1,THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA\nBài: THỜI NG...,lesson,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,None
2,Những dấu tích của Người tối cổ được tìm thấy ...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nNhững ...,title,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,None
3,"Ở giai đoạn đầu, Người tinh khôn sống như thế ...",Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nỞ giai...,title,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,None
4,Giai đoạn phát triển của Người tinh khôn có gì...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nGiai đ...,title,Thực hiện kế hoạch 5 năm 1996 - 2000 Bài: Đất ...,None


# Load tokenizer model

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    "trituenhantaoio/bert-base-vietnamese-uncased",
    cache_dir="../../tokenizer_models/"
)

# Chungking data về kích thước 512 tokens

In [ ]:
def chunk_text(text, max_length=512, max_tokens=512, stride=64, tokenizer=None):
    """Split text into chunks of max_length tokens."""
    if text is None:
        raise ValueError("Text must be provided.")
    if max_length > 512:
        raise ValueError("max_length must be less than or equal to 512.")
    if max_tokens > 512:
        raise ValueError("max_tokens must be less than or equal to 512.")
    if tokenizer is None:
        raise ValueError("Tokenizer must be provided.")
    
    tokens = tokenizer.encode(text, add_special_tokens=True)
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + max_tokens
        chunk = tokens[start:end]
        chunks.append(chunk)

        # Nếu có stride (chồng lấn token giữa các đoạn)
        start += max_tokens - stride if stride > 0 else max_tokens

    # Decode lại các đoạn thành văn bản
    chunk_texts = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]
    return chunk_texts

In [21]:
text = "Đây là một đoạn văn bản tiếng Việt rất dài..." * 500
chunks = chunk_text(text, max_tokens=512, stride=100, tokenizer=tokenizer)

for i, chunk in enumerate(chunks):
    print(f"--- Chunk {i+1} ---")
    print(chunk[:200], "...")  # In thử 200 ký tự đầu


--- Chunk 1 ---
đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tien ...
--- Chunk 2 ---
tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mo ...
--- Chunk 3 ---
đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tien ...
--- Chunk 4 ---
ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay l ...
--- Chunk 5 ---
đay la mot đoan van ban tieng viet rat dai đay la mot đoan van ban tieng viet rat dai đay la mot đoa